# Piping Chains and the RunnablePassthrough Class

In [ ]:
# Run the line of code below to check the version of langchain in the current environment.
# Substitute "langchain" with any other package name to check their version.

In [1]:
pip show langchain

Name: langchain
Version: 1.0.7
Summary: Building applications with LLMs through composability
Home-page: https://docs.langchain.com/
Author: 
Author-email: 
License: MIT
Location: /opt/anaconda3/envs/LangChain_course_env1/lib/python3.13/site-packages
Requires: langchain-core, langgraph, pydantic
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [2]:
import sys
sys.path.append("..")
import sks_config

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [4]:
RunnablePassthrough().invoke([1, 2, 3])

[1, 2, 3]

In [5]:
chat_template_tools = ChatPromptTemplate.from_template('''
What are the five most important tools a {job title} needs?
Answer only by listing the tools.
''')

chat_template_strategy = ChatPromptTemplate.from_template('''
Considering the tools provided, develop a strategy for effectively learning and mastering them:
{tools}
''')

In [6]:
chat_template_tools

ChatPromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['job title'], input_types={}, partial_variables={}, template='\nWhat are the five most important tools a {job title} needs?\nAnswer only by listing the tools.\n'), additional_kwargs={})])

In [7]:
chat = ChatOpenAI(api_key = sks_config.SKS_OPENAI_API_KEY,
                  model_name = 'gpt-4o-mini', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 100)

/opt/anaconda3/envs/LangChain_course_env1/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3639: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [8]:
string_parser = StrOutputParser()

In [9]:
chain_tools = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()})
chain_strategy = chat_template_strategy | chat | string_parser

In [10]:
print(chain_tools.invoke({'job title':'data scientist'}))

{'tools': '1. Python\n2. R\n3. SQL\n4. Jupyter Notebook\n5. Git'}


In [11]:
print(chain_strategy.invoke({'tools':'''
1. Python
2. R Programming
3. SQL
4. Tableau
5. Hadoop
'''}))

To effectively learn and master the tools you've listed—Python, R Programming, SQL, Tableau, and Hadoop—it's essential to develop a structured strategy that incorporates various learning methods, practice, and real-world application. Here’s a comprehensive strategy:

### 1. Set Clear Goals
- **Define Objectives**: Determine what you want to achieve with each tool (e.g., data analysis, data visualization, big data processing).
- **Timeline**: Set a realistic timeline for mastering each tool based on


In [12]:
chain_combined = chain_tools | chain_strategy

In [13]:
print(chain_combined.invoke({'job title':'data scientist'}))

To effectively learn and master Python, R, SQL, Jupyter Notebook, and Git, you can follow a structured strategy that incorporates a mix of theoretical understanding, practical application, and consistent practice. Here’s a step-by-step approach:

### 1. Set Clear Goals
- **Define Objectives**: Determine what you want to achieve with each tool. For example, do you want to analyze data, build machine learning models, or manage version control for projects?
- **Create a Timeline**:


In [14]:
chain_long = (chat_template_tools | chat | string_parser | {'tools':RunnablePassthrough()} | 
              chat_template_strategy | chat | string_parser)

In [15]:
chain_long.get_graph().print_ascii()

     +-------------+       
     | PromptInput |       
     +-------------+       
            *              
            *              
            *              
  +--------------------+   
  | ChatPromptTemplate |   
  +--------------------+   
            *              
            *              
            *              
      +------------+       
      | ChatOpenAI |       
      +------------+       
            *              
            *              
            *              
   +-----------------+     
   | StrOutputParser |     
   +-----------------+     
            *              
            *              
            *              
+-----------------------+  
| StrOutputParserOutput |  
+-----------------------+  
            *              
            *              
            *              
     +-------------+       
     | Passthrough |       
     +-------------+       
            *              
            *              
            *       